In [ ]:
import prefect
from prefect import task, Flow

import requests
from unittest.mock import MagicMock, patch
from time import sleep


@task
def create_payload():
    "Performs expensive computation to create / return an URL"

    sleep(5) # for whatever reason, getting to this point takes a long time
    return 'http://www.google.com'


@task(max_retries=1)
def ping_external_service(url):
    "Performs a simple GET request to the provided URL, and returns the text of the response."

    if prefect.context.get("_fail"):
        raise ValueError("Request failed with status code 418.")
    else:
        r = requests.get(url)
        return r.text


In [ ]:
with Flow(name="retry example") as f:
    text = ping_external_service(create_payload())


In [ ]:
%%time
with prefect.context(_fail=True):
    flow_state = f.run(return_tasks=f.tasks)

##    CPU times: user 5.65 ms, sys: 1.46 ms, total: 7.12 ms
##    Wall time: 5.01 s


In [ ]:
print("Flow state: {}\n".format(flow_state))
print("Flow results: {}".format(flow_state.result))

## Flow state: Pending("Some terminal tasks are still pending.")

## Flow results: {
##      <Task: create_payload>: Success("Task run succeeded."),
##      <Task: ping_external_service>: Retrying("Retrying Task (after attempt 1 of 2)")
##                }


In [ ]:
%%time
new_flow_state = f.run(return_tasks=[text],
                       task_states={text: flow_state.result[text]},
                       start_tasks=[text])

##    CPU times: user 15.5 ms, sys: 5.91 ms, total: 21.4 ms
##    Wall time: 167 ms


In [ ]:
print("Flow state: {}\n".format(new_flow_state))
print("Flow results: {}".format(new_flow_state.result))

##     Flow state: Success("All reference tasks succeeded.")

##     Flow results: {<Task: ping_external_service>: Success("Task run succeeded.")}
